# Milestone Project: Blackjack

- Player starts with 2 cards face up
- Dealer starts with 1 card face down and 1 face up
- Player goes first in gameplay
- Player goal: Get closer to a total value of 21 than the dealer does. Possible actions: (Hit and Stay)
- After player turn: If player is under 21, dealer then hits until more or equal 17 points
- Sepcial Rules: Face Cards (Jack, Queen, King) count as a value of 10 and Aces count as either 1 or 11 (whichever player picks)

In [54]:
from random import shuffle as random_shuffle
from IPython.display import clear_output

In [55]:
class Card():
    
    def __init__(self, suit, rank, value):
        self.suit = suit
        self.rank = rank
        self.value = value
        
    def __str__(self):
        return f"{self.suit} {self.rank} ({self.value})"
    
    def __repr__(self):
        return str(self)

In [56]:
class Deck(list):
    
    suits = ('Hearts', 'Diamonds', 'Spades', 'Clubs')
    values = {'Two': 2, 'Three': 3, 'Four': 4, 'Five': 5, 'Six': 6, 'Seven': 7, 'Eight': 8, 'Nine': 9, 'Ten': 10, 'Jack': 10, 'Queen': 10, 'King': 10, 'Ace': 11}
    
    def __init__(self):
        super().__init__(Card(suit, rank, value) for suit in Deck.suits for rank, value in Deck.values.items())
        self.shuffle()
        
    def get_n_cards(self, n):
        if len(self) >= n:
            self.__init__()
        return [self.pop() for _ in range(n)]
            
    
    def shuffle(self):
        random_shuffle(self)

In [57]:
class Hand(list):
    
    def __init__(self, *cards):
        super().__init__(card for card in cards)
    
    def add_cards(self, cards):
        self.extend(cards)
        
    def throw(self):
        self.clear()
        
    def get_sum(self):
        return sum(card.value for card in self)
        
    def __str__(self):
        return ', '.join(str(card) for card in self)

In [58]:
class Player():
    
    def __init__(self, name, balance):
        self.name = name
        self.balance = balance
        self.hand = Hand()
        
    def deposit(self, amount):
        self.balance += amount
        
    def withdraw(self, amount):
        self.balance -= amount
        
    def add_cards_to_hand(self, card):
        self.hand.add_cards(card)
        
    def throw_hand(self):
        self.hand.throw()
        
    def get_hand_sum(self):
        return self.hand.get_sum()
        
    def __str__(self):
        return f"{self.name} with {self.balance}€"

In [59]:
class NotEnoughMoneyError(Exception):
    def __init__(self):
        super().__init__("You do not have enough money!")

In [60]:
def get_players_information():
    player_name = input("Enter your name: ")
    while True:
        try:
            player_balance = float(input("Enter your balance: "))
        except ValueError:
            print("Please enter a valid balance!")
            continue
        else:
            break
    return Player(player_name, player_balance)

In [61]:
def get_player_round_deposit(player):
    while True:
        try:
            amount = int(input("With how much money do you want to play this round? "))
            if amount > player.balance:
                raise NotEnoughMoneyError
        except ValueError:
            print("Please enter a valid amount!")
        except NotEnoughMoneyError as err:
            print(err)
        else:
            break
    return amount

In [62]:
def get_player_action():
    return input("Hit or stay? h/s ").lower().startswith("h")

In [63]:
def replay():
    return input("Do you want to play another round? Y/n").lower().startswith('y')

In [64]:
def replay_with_deposit():
    return input("You don't have enough money for another round. Do you want to deposit new money? Y/n").lower().startswith('y')

In [ ]:
print("Welcome to Blackjack!")
player = get_players_information()
dealer = Player("Dealer", 0)
deck = Deck()

while True:
    player_deposit = get_player_round_deposit(player)
    clear_output()
    player.withdraw(player_deposit)
    player.add_cards_to_hand(deck.get_n_cards(2))
    dealer.add_cards_to_hand(deck.get_n_cards(2))
    
    print(f"The dealers first card is {dealer.hand[0]}")
    
    while True:
        print(f"{player.name}, your current hand is: {player.hand}\n")
        if get_player_action():
            new_card = deck.get_n_cards(1)
            print(f"You pulled {new_card[0]}")
            player.add_cards_to_hand(new_card)
            if player.get_hand_sum() <= 21:
                continue
            else:
                break
        else:
            break
            
    if player.get_hand_sum() <= 21:
        
        print(f"\nThe dealer hand is {dealer.hand}")
        
        while dealer.get_hand_sum() < 17:
            new_card = deck.get_n_cards(1)
            print(f"The dealer pulled {new_card[0]}")
            dealer.add_cards_to_hand(new_card)
            
            
        print(f"\nYour hand value is {player.get_hand_sum()}")
        print(f"The dealers hand value is {dealer.get_hand_sum()}")
            
        if dealer.get_hand_sum() > 21 or dealer.get_hand_sum() < player.get_hand_sum():
            player.deposit(player_deposit * 2)
            print(f"You have won! Balance: {player.balance}")
        else:
            if dealer.get_hand_sum() > player.get_hand_sum():
                print(f"Sorry, you have lost! Balance: {player.balance}")
            else:
                player.deposit(player_deposit)
                print(f"Draw! Balance: {player.balance}")
                
    else:
        print(f"Sorry, you have lost! Balance: {player.balance}")
        
    if not replay():
        break
    else:
          player.throw_hand()
          dealer.throw_hand()

print("Thank you for playing!")

The dealers first card is Diamonds Five (5)
Manuel, your current hand is: Diamonds Four (4), Diamonds Six (6)

Hit or stay? h/s h
You pulled Spades Nine (9)
Manuel, your current hand is: Diamonds Four (4), Diamonds Six (6), Spades Nine (9)

